* [Company Summary Variables List](https://api.census.gov/data/2019/abscs/variables.html) <br/><br/>
* [Company Summary Documentation](https://www2.census.gov/programs-surveys/abs/technical-documentation/api/API2019-company-summary-1-26-2021.pdf)

In [46]:
# US CENSUS DATA
import pandas as pd

# MAKE SURE THERE ARE NO WHITE SPACES IN ANY VARIABLE
labels = 'RACE_GROUP,RACE_GROUP_LABEL,EMP,YEAR'

data_group = '&for=us'  # FOR ENTIRE USA
# data_group = '&for=metropolitan%20statistical%20area/micropolitan%20statistical%20' # FOR METROPOLITAN AREAS
# data_group = '&for=state' # FOR STATES

# Set area = '' for USA and US STATES
# area = 'area:12060'
area = ''


NAICS2017_value = ''

API_key = '&key=1ba63b3df04e1bd07755febe18f836df1b86fc9f'

df = pd.read_json(f'https://api.census.gov/data/2018/abscs?get={labels}{data_group}{area}{NAICS2017_value}{API_key}')
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data after the header row
df.columns = new_header #set the header row as the df header
df

,RACE_GROUP,RACE_GROUP_LABEL,EMP,YEAR,us
1,00,Total,128196406,2018,1
2,40,Black or African American,1188819,2018,1
3,30,White,56877432,2018,1
4,50,American Indian and Alaska Native,200256,2018,1
5,60,Asian,5090065,2018,1
6,90,Minority,9432081,2018,1
7,70,Native Hawaiian and Other Pacific Islander,54446,2018,1
8,91,Equally minority/nonminority,843404,2018,1
9,92,Nonminority,53190299,2018,1
